In [3]:
# Computation of GW invariants of all rational complex surfaces, 
# i.e. P^1*P^1 and P_k the blown-up plane in k points.
import math
import scipy.special

def minus_list(l1,l2):
    return [x for x in l1 if x not in l2]

def binom(n,k):
    return 0 if n <0 or k>n else scipy.special.binom(n,k)

## GW of P^1*P^1
## These invariants are stored in the variable NP1P1
NP1P1 = dict()
NP1P1[(1,0)] = NP1P1[(0,1)] = 1
## One could also use the next GWPk(d,i,d-i]) instead, but this one seems to be faster.
def GWP1P1(d):
    a = max(d[0],d[1])
    b = min(d[0],d[1])
 # cheked that this value has not been computed before
    if not (a,b) in NP1P1.keys():
        if 2*(a+b)-1<0:
            NP1P1[(a,b)] = 0
        else:
            n = 0
            for aa in range(a+1):
                for bb in range(b+1):
                    if (aa!=0 or bb!=0) and (aa!=a or bb!=b):
                        n += GWP1P1([max(aa,bb),min(aa,bb)])*GWP1P1([max(a-aa,b-bb),min(a-aa,b-bb)])*(aa*(b-bb)+(a-aa)*bb)*(b-bb)*(aa*binom(2*(a+b)-4,2*(aa+bb)-2)-(a-aa)*binom(2*(a+b)-4,2*(aa+bb)-3))
            NP1P1[(a,b)] = n
    NP1P1[(b,a)] = NP1P1[(a,b)]
    return NP1P1[(a,b)]

# GW of P^2 blown up in k points basis used for the 2nd homology group: L (the line class) and E_i (the ith exceptional class)
# we write [d,[c_1, ..., c_{k}]] for dL - sum(c_i E_i) these invariants are stored in the variable NPk 
# first some procedures ordonate a list of non-negative integer from the highest to the lowest term

def orderlist(l):
    return sorted(l,reverse=True) 

# compute all possible partitions of [d,[c_i]]

def part(d,al):
    parti = []
    for d1 in range(1,d):
        ll = part2(d,d1,al)
        for i in range(len(ll)):
            parti.append([d1,ll[i]])
    return parti

def part2(d,d1,l):
    if len(l) == 0:
        return [[]]
    else:
        lll = part2(d,d1,l[1:])
        ll = []
        for i in range(min(l[0],d1)):
            if l[0] - i <= d-d1:
                for j in range(len(lll)):
                    if genus(d1,[i,lll[j]]) >= 0 and genus(d-d1,minus_list(l,[i,lll[j]])) >=0:
                        return ll.extend([i,lll[j]])
def genus(d,al):
    return (d-1)*(d-2)/2 - sum([item*(item-1)/2 for item in al])

NPk = dict()
NPk[(1,())] = 1
NPk[(0,(-1))] = 1

# GW invariants of P^2 blown up in k points
# d is the degree, all contains intersection numbers with exceptional divisors
# if all is missing, it is understood as all=[]

def GWPk(d,all):
    all = list(all)  
    npoints = 3*d - sum([item for item in all]) - 1
    if genus(d,all) <0 or npoints <0:
        al = [item for item in all]
        NPk[(d,tuple(al))] = 0
    elif min(all) == -1:
        if d != 0 or max(all) >0 or (sum(all) < -1):
            al = [item for item in all]
            NPk[(d,tuple(al))] = 0
        else:
            al = [-1]
    else:
        al = orderlist(all)
    if (d,tuple(al)) in NPk.keys():
        npoints = 3*d-sum(al) - 1
        if npoints >= 3:
            parti = part(d,al)
            n = 0
            for i in range(len(parti)):
                d1 = parti[i][0]
                d2 = d-d1
                be = parti[i][1]
                ga = minus_list(al,be)
                n1 = 3*d1-sum(be) - 1
                n += GWPk(d1,be)*GWPk(d2,ga)*(d1*d2-sum([be[j]*ga[j] for j in range(len(be))])) * (d1*d2*binom(npoints-3,n1-1)-d1**2*binom(npoints-3,n1))
            NPk[(d,tuple(al))] = n 
        elif npoints >= 0:
            al[0] = al[0]-1
            parti = part(d,al)
            n = (d**2-al[0]**2)*GWPk(d,al)
            for i in range(len(parti)):
                d1 = parti[i][0]
                d2 = d-d1
                be = parti[i][1]
                ga = minus_list(al,be)
                n1 = 3*d1 - sum(be) - 1
                n += GWPk(d1,be) *GWPk(d2,ga) * (d1*d2 - sum([be[j]*ga[j] for j in range(len(be))])) *(d1*d2*be[0]*ga[0]-d1**2*ga[0]**2)*binom(npoints,n1)
            al[1] += 1
            NPk[(d,tuple(al))] = n/(d**2*al[0])
        else:
            NPk[(d,tuple(al))] = 0
    return NPk[(d,tuple(al))]

0
